In [ ]:
import os
os.environ["GST_PLUGIN_PATH"] = "/usr/local/Cellar/gstreamer/1.22.4/lib/gstreamer-1.0" + os.pathsep + os.environ.get("GST_PLUGIN_PATH", "")
print (os.environ["GST_PLUGIN_PATH"])

In [ ]:
!find /usr/local/Cellar/gstreamer/1.22.4/lib/gstreamer-1.0 -name "uridecodeuri.so"

In [ ]:
!gst-launch-1.0 videotestsrc ! fakesink
!gst-inspect-1.0 uridecodeuri  # Check that uridecodeuri is found


In [ ]:
import os
import gi

gi.require_version('Gst', '1.0')
from gi.repository import Gst, GLib

# Initialize GStreamer
Gst.init(None)

# Optional: Set GST_PLUGIN_PATH (if needed) - adjust the path if necessary
# os.environ["GST_PLUGIN_PATH"] = "/usr/local/Cellar/gstreamer/1.22.4/lib/gstreamer-1.0" + os.pathsep + os.environ.get("GST_PLUGIN_PATH", "")
#print (os.environ["GST_PLUGIN_PATH"]) # see the path

def create_pipeline(input_uri, video_output_file, audio_output_file):
    """
    Creates a GStreamer pipeline to split video and audio.
    """
    pipeline_string = f"""
        uridecodeuri uri="{input_uri}" ! tee name=t
        t. ! queue ! matroskamux ! filesink location="{video_output_file}"
        t. ! queue ! audioconvert ! vorbisenc ! oggmux ! filesink location="{audio_output_file}"
    """
    try:
        pipeline = Gst.parse_launch(pipeline_string)
        return pipeline
    except GLib.Error as e:
        print(f"Error creating pipeline: {e}")
        return None


def bus_call(bus, message, loop):
    """
    Callback function to handle messages from the GStreamer bus.
    """
    t = message.type
    if t == Gst.MessageType.EOS:
        print("End-of-stream")
        loop.quit()
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        print(f"Error: {err}, {debug}")
        loop.quit()
    return GLib.SOURCE_CONTINUE


def process_video(input_file_path, output_folder):
    """
    Processes a single video file, extracting video and audio.
    """
    file_name = os.path.basename(input_file_path)
    file_name_without_ext, file_ext = os.path.splitext(file_name)

    video_output_file = os.path.join(output_folder, f"{file_name_without_ext}_video.mkv")  # Matroska for video
    audio_output_file = os.path.join(output_folder, f"{file_name_without_ext}_audio.ogg")  # Ogg for audio with Vorbis

    pipeline = create_pipeline(f"file://{input_file_path}", video_output_file, audio_output_file)

    if not pipeline:
        print(f"Failed to create pipeline for {input_file_path}")
        return

    loop = GLib.MainLoop()
    bus = pipeline.get_bus()
    bus.add_signal_watch()
    bus.connect("message", bus_call, loop)

    pipeline.set_state(Gst.State.PLAYING)

    try:
        loop.run()
    except KeyboardInterrupt:
        print("Interrupted by user")
    finally:
        pipeline.set_state(Gst.State.NULL)
        bus.remove_signal_watch()


def traverse_directory(root_directory, output_root_folder):
    """
    Traverses a directory and processes all video files found within.
    """
    for foldername, subfolders, filenames in os.walk(root_directory):
        # Create the corresponding output folder
        relative_path = os.path.relpath(foldername, root_directory)
        output_folder = os.path.join(output_root_folder, relative_path)
        os.makedirs(output_folder, exist_ok=True)  # Create the folder

        for filename in filenames:
            if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv', '.webm')):  # Check video extensions
                input_file_path = os.path.join(foldername, filename)
                print(f"Processing: {input_file_path}")
                process_video(input_file_path, output_folder)
                print(f"Finished Processing: {input_file_path}")



if __name__ == '__main__':
    input_directory = "/path/to/your/video/directory"  # Replace with your input directory
    output_directory = "/path/to/your/output/directory"  # Replace with your output directory

    traverse_directory(input_directory, output_directory)

In [ ]:
import cv2
print(cv2.__version__)
print(cv2.getBuildInformation())

In [ ]:
import site
print(site.getsitepackages())


In [3]:
!gst-inspect-1.0

accurip:  accurip: AccurateRip(TM) CRC element
adaptivedemux2:  dashdemux2: DASH Demuxer
adaptivedemux2:  hlsdemux2: HLS Demuxer
adaptivedemux2:  mssdemux2: Smooth Streaming demuxer (v2)
adder:  adder: Adder
adpcmdec:  adpcmdec: ADPCM decoder
adpcmenc:  adpcmenc: ADPCM encoder
aes:  aesdec: aesdec
aes:  aesenc: aesenc
aiff:  aiffmux: AIFF audio muxer
aiff:  aiffparse: AIFF audio demuxer
alaw:  alawdec: A Law audio decoder
alaw:  alawenc: A Law audio encoder
alpha:  alpha: Alpha filter
alphacolor:  alphacolor: Alpha color filter
amrnb:  amrnbdec: AMR-NB audio decoder
amrnb:  amrnbenc: AMR-NB audio encoder
amrwbdec:  amrwbdec: AMR-WB audio decoder
analyticsoverlay:  objectdetectionoverlay: Object Detection Overlay
aom:  av1dec: AV1 Decoder
aom:  av1enc: AV1 Encoder
apetag:  apedemux: APE tag demuxer
app:  appsink: AppSink
app:  appsrc: AppSrc
applemedia:  avfassetsrc: Source and decoder for AVFoundation assets
applemedia:  avfdeviceprovider (GstDeviceProviderFactory)
applemedia:  avfvide

In [8]:
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst, GObject, GLib

def split_video_audio(input_file, audio_output_file, video_output_file):
    """
    Splits an MP4 video file into separate audio and video files using GStreamer.

    Args:
        input_file (str): Path to the input MP4 video file.
        audio_output_file (str): Path to the output audio file (e.g., "audio.mp3").  Must have a file extension suitable for encoding.
        video_output_file (str): Path to the output video file (e.g., "video.mp4"). Must have a file extension suitable for encoding.
    """

    Gst.init(None)

    # Create the pipeline
    pipeline = Gst.Pipeline()

    # Create elements
    source = Gst.ElementFactory.make("filesrc", "source")
    demuxer = Gst.ElementFactory.make("qtdemux", "demuxer") # For MP4 files
    audio_queue = Gst.ElementFactory.make("queue", "audio_queue")
    video_queue = Gst.ElementFactory.make("queue", "video_queue")

    # Audio elements
    audio_convert = Gst.ElementFactory.make("audioconvert", "audio_convert")
    audio_encoder = None # Choose an audio encoder based on output file extension
    if audio_output_file.endswith(".mp3"):
        audio_encoder = Gst.ElementFactory.make("lamemp3enc", "audio_encoder")
    elif audio_output_file.endswith(".ogg"):
        audio_encoder = Gst.ElementFactory.make("vorbisenc", "audio_encoder")
        audio_encoder.set_property("quality", 0.4)  # Adjust quality as needed
    elif audio_output_file.endswith(".aac"):
        audio_encoder = Gst.ElementFactory.make("faac", "audio_encoder")  # requires faac package.  Alternative: avenc_aac
        #audio_encoder = Gst.ElementFactory.make("avenc_aac", "audio_encoder")
    elif audio_output_file.endswith(".wav"):
        audio_encoder = Gst.ElementFactory.make("wavenc", "audio_encoder") # Not really an encoder but puts it in wave format
    else:
        raise ValueError("Unsupported audio output file extension.  Try .mp3, .ogg, .aac, or .wav")


    audio_sink = Gst.ElementFactory.make("filesink", "audio_sink")

    # Video elements
    video_convert = Gst.ElementFactory.make("videoconvert", "video_convert")
    video_encoder = None  # Choose a video encoder based on output file extension
    if video_output_file.endswith(".mp4"):
         video_encoder = Gst.ElementFactory.make("x264enc", "video_encoder")
         video_encoder.set_property("tune", "zerolatency") #For low latency encoding
         video_encoder.set_property("pass", 5)   # Increase quality, higher number better quality.  Defaults to 0
         video_encoder.set_property("bitrate", 2048) # Bitrate in kbps
         mp4mux = Gst.ElementFactory.make("mp4mux", "mp4mux")
    elif video_output_file.endswith(".webm"):
        video_encoder = Gst.ElementFactory.make("vp8enc", "video_encoder") #Or vp9enc for better encoding, but is more resource intensive
        muxer = Gst.ElementFactory.make("webmmux", "webmmux")
    elif video_output_file.endswith(".avi"):
         video_encoder = Gst.ElementFactory.make("x264enc", "video_encoder")
         muxer = Gst.ElementFactory.make("avimux", "avimux")

    else:
        raise ValueError("Unsupported video output file extension. Try .mp4, .webm or .avi")


    video_sink = Gst.ElementFactory.make("filesink", "video_sink")



    # Set properties
    source.set_property("location", input_file)
    audio_sink.set_property("location", audio_output_file)
    video_sink.set_property("location", video_output_file)



    # Add elements to the pipeline
    elements = [source, demuxer, audio_queue, video_queue, audio_convert, audio_encoder, audio_sink, video_convert, video_encoder, video_sink]
    if video_output_file.endswith(".mp4"):
        elements.append(mp4mux)
        pipeline.add(source, demuxer, audio_queue, video_queue, audio_convert, audio_encoder, audio_sink, video_convert, video_encoder, mp4mux, video_sink)
    else:
        elements.append(muxer)
        pipeline.add(source, demuxer, audio_queue, video_queue, audio_convert, audio_encoder, audio_sink, video_convert, video_encoder, muxer, video_sink)


    # Link elements
    source.link(demuxer)
    demuxer.connect("pad-added", on_pad_added, audio_queue, video_queue)

    #Audio branch
    audio_queue.link(audio_convert)
    audio_convert.link(audio_encoder)
    audio_encoder.link(audio_sink)

    #Video branch
    video_queue.link(video_convert)
    video_convert.link(video_encoder)
    if video_output_file.endswith(".mp4"):
        video_encoder.link(mp4mux)
        mp4mux.link(video_sink)
    else:
        video_encoder.link(muxer)
        muxer.link(video_sink)

    # Create bus and set callbacks
    bus = pipeline.get_bus()
    bus.add_signal_watch()
    bus.connect("message::eos", on_eos, pipeline)
    bus.connect("message::error", on_error, pipeline)

    # Start the pipeline
    pipeline.set_state(Gst.State.PLAYING)

    # Run the main loop
    loop = GLib.MainLoop()
    try:
        loop.run()
    except KeyboardInterrupt:
        print("Interrupted")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()



def on_pad_added(demuxer, pad, audio_queue, video_queue):
    """
    Callback function for when a pad is added to the demuxer.  Connects the correct stream to the appropriate queue.
    """
    padname = pad.get_name()
    if padname.startswith("audio"):
        queue = audio_queue
    elif padname.startswith("video"):
        queue = video_queue
    else:
        print(f"Unknown pad {padname}, ignoring.")
        return

    sinkpad = queue.get_static_pad("sink")
    if sinkpad.is_linked():
        print(f"Pad {sinkpad.get_name()} is already linked, ignoring.")
        return

    new_pad_type = pad.query_caps(None).to_string()
    print(f"Linking demuxer pad {pad.get_name()} of type {new_pad_type} to queue {queue.get_name()}")

    ret = pad.link(sinkpad)
    if ret != Gst.PadLinkReturn.OK:
        print(f"Type is {new_pad_type}")
        print("Pad link failed:", ret)


def on_eos(bus, message, pipeline):
    """
    Callback function for when the end of stream is reached.
    """
    print("End of stream")
    pipeline.set_state(Gst.State.NULL)
    GLib.MainLoop().quit()

def on_error(bus, message, pipeline):
    """
    Callback function for when an error occurs.
    """
    err, debug_info = message.parse_error()
    print("Error received from element %s: %s" % (message.src.get_name(), err))
    print("Debugging information: %s" % debug_info)
    pipeline.set_state(Gst.State.NULL)
    GLib.MainLoop().quit()


if __name__ == '__main__':
    input_file = "input.mp4"  # Replace with your input file
    audio_output_file = "audio.mp3"  # Replace with your desired audio output file
    video_output_file = "video.mp4"  # Replace with your desired video output file

    # Create a dummy input file if it doesn't exist, to prevent crashes.
    import os
    if not os.path.exists(input_file):
        print(f"Warning:  Input file {input_file} does not exist.  Creating a dummy file.")
        open(input_file, 'a').close()


    try:
        split_video_audio(input_file, audio_output_file, video_output_file)
    except ValueError as e:
        print(f"Error: {e}")
        print("Please check your output file extensions and ensure the necessary codecs are installed.")

GError: gi-invoke-error-quark: Could not locate g_option_error_quark: dlopen(libglib-2.0.0.dylib, 0x0009): tried: 'libglib-2.0.0.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibglib-2.0.0.dylib' (no such file), '/usr/lib/libglib-2.0.0.dylib' (no such file, not in dyld cache), 'libglib-2.0.0.dylib' (no such file) (1)

In [ ]:
export GST_VERSION=1.0  # Or the appropriate version if not 1.0
export PATH="/opt/homebrew/Cellar/gstreamer/1.26.0_1/bin:$PATH"
export PKG_CONFIG_PATH="/opt/homebrew/Cellar/gstreamer/1.26.0_1/lib/pkgconfig:$PKG_CONFIG_PATH"
export LD_LIBRARY_PATH="/opt/homebrew/Cellar/gstreamer/1.26.0_1/lib:$LD_LIBRARY_PATH"
export GI_TYPELIB_PATH="/opt/homebrew/Cellar/gstreamer/1.26.0_1/share/gir-1.0:$GI_TYPELIB_PATH"

In [11]:
import gi
gi.require_version('GLib', '2.0')
from gi.repository import GLib

print("GLib version:", GLib.MAJOR_VERSION, GLib.MINOR_VERSION, GLib.MICRO_VERSION)

GError: gi-invoke-error-quark: Could not locate g_option_error_quark: dlopen(libglib-2.0.0.dylib, 0x0009): tried: 'libglib-2.0.0.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibglib-2.0.0.dylib' (no such file), '/usr/lib/libglib-2.0.0.dylib' (no such file, not in dyld cache), 'libglib-2.0.0.dylib' (no such file) (1)

In [12]:
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst, GLib

def check_gstreamer():
  """Checks if GStreamer is installed and functional."""

  try:
    Gst.init(None)
    print("GStreamer is installed and initialized correctly.")

    # Check version
    version = Gst.version_string()
    print(f"GStreamer version: {version}")

    # Check if a basic element can be created (indicates core functionality)
    try:
      source = Gst.ElementFactory.make("fakesrc", "source")
      if source:
        print("Basic element creation successful.")
        source.unparent() # Cleanup element

      else:
          print("Failed to create a basic GStreamer element (fakesrc). Possible installation issue.")
          return False
    except Exception as e:
      print(f"Error creating GStreamer element: {e}")
      print("Possible installation issue or missing plugins.")
      return False

    return True  # Everything seems OK

  except Exception as e:
    print(f"GStreamer initialization failed: {e}")
    print("GStreamer might not be installed or not properly configured.")
    return False


if __name__ == "__main__":
  if check_gstreamer():
    print("GStreamer check passed.")
  else:
    print("GStreamer check failed.")

GError: gi-invoke-error-quark: Could not locate g_option_error_quark: dlopen(libglib-2.0.0.dylib, 0x0009): tried: 'libglib-2.0.0.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibglib-2.0.0.dylib' (no such file), '/usr/lib/libglib-2.0.0.dylib' (no such file, not in dyld cache), 'libglib-2.0.0.dylib' (no such file) (1)